https://github.com/ageron/handson-ml2/blob/master/09_unsupervised_learning.ipynb

In [1]:
# Q11 - Chapter 9 Unsupervised learning
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt

In [2]:
olivetti = fetch_olivetti_faces()

In [3]:
X = olivetti.data
y = olivetti.target

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
strat_split = StratifiedShuffleSplit(n_splits=1, test_size =40, random_state=42) #n_spliits = 1???

# This first loop seperates the data set into two - X train AND valid + X_test 
# So really only gives X_test. X_train_valid will be used in the next loop to further separate train and valid
for train_valid_idx, test_idx in strat_split.split(X, y):
    X_train_valid, X_test = X[train_valid_idx], X[test_idx]
    y_train_valid, y_test = y[train_valid_idx], y[test_idx]

# further splits X_train_valid into train + valid
strat_split = StratifiedShuffleSplit(n_splits=1, test_size =80, random_state=42)
for train_idx, valid_idx in strat_split.split(X_train_valid, y_train_valid):
    X_train, X_valid = X_train_valid[train_idx], X_train_valid[valid_idx]
    y_train, y_valid = y_train_valid[train_idx], y_train_valid[valid_idx]

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(280, 4096) (280,)
(80, 4096) (80,)
(40, 4096) (40,)


In [16]:
from sklearn.decomposition import PCA
pca = PCA(0.99)
X_train_pca = pca.fit_transform(X_train)
X_valid_pca = pca.transform(X_valid)
X_test_pca = pca.transform(X_test)

# Note in the above code, the train/valid/test has been made, then PCA is used after.
# Do a fit_transform on the training data, then ONLY a transform on your valid and test set.

print(X_train_pca.shape, y_train.shape)
print(X_valid_pca.shape, y_valid.shape)
print(X_test_pca.shape, y_test.shape)
pca.n_components_

(280, 200) (280,)
(80, 200) (80,)
(40, 200) (40,)


200

In [19]:
# Train a classifier using RandomForest

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
clf.score(X_valid, y_valid)

0.9375

In [20]:
X_train_pca.shape

(280, 200)

In [21]:
clf2 = RandomForestClassifier(random_state=42)
clf2.fit(X_train_pca, y_train)
clf2.score(X_valid_pca, y_valid)

0.8875

In [22]:
# Use KMeans to reduce dimensionality even further

from sklearn.cluster import KMeans

# from kmeans_1 the best number of clusters was k= 105
best_model = KMeans(n_clusters=105, random_state=42).fit(X_train_pca)

In [23]:
X_train_reduced = best_model.transform(X_train_pca)
X_valid_reduced = best_model.transform(X_valid_pca)
X_test_reduced = best_model.transform(X_test_pca)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_reduced, y_train)
    
clf.score(X_valid_reduced, y_valid)

0.7375